In [1]:
# https://huggingface.co/datasets/openslr/librispeech_asr

# https://huggingface.co/openai/whisper-tiny.en

# Load model directly
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import torch
import torchaudio
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq


/home/darrell/Documents/GitHub/ToolHub/venv_test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


processor = AutoProcessor.from_pretrained("openai/whisper-tiny.en")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-tiny.en")

# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
print(transcription)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.']
[' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.']


In [3]:
# Load your own audio file (ensure it's in a supported format like WAV)
audio_path = "./testAudio.wav"  # Replace with your actual file path

# Load the audio using torchaudio
waveform, sample_rate = torchaudio.load(audio_path)

# Whisper expects a 16kHz sample rate, resample if necessary
target_sample_rate = 16000
if sample_rate != target_sample_rate:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
    waveform = resampler(waveform)

# Convert waveform to numpy array and process it
input_features = processor(waveform.squeeze().numpy(), sampling_rate=target_sample_rate, return_tensors="pt").input_features

# Generate transcription
with torch.no_grad():
    predicted_ids = model.generate(input_features)

# Decode the output to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print("Transcription:", transcription[0])


Transcription:  I repeat, hello. This message lets you know that you did it. Please call us for further assistance. Thank you. Thank you. Have a great day.
